In [ ]:
# Test upload

In [6]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import plotly.express as px               #to create interactive charts
import plotly.graph_objects as go         #to create interactive charts
from jupyter_dash import JupyterDash      #to build Dash apps from Jupyter environments
import dash_core_components as dcc        #to get components for interactive user interfaces
import dash_html_components as html       #to compose the dash layout using Python structures
import webbrowser
from dash import Dash, html, Input, Output, State, dash_table
import nbconvert
import base64
import io


In [11]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets,
                suppress_callback_exceptions=True)

app.layout = html.Div([ # this code section taken from Dash docs https://dash.plotly.com/dash-core-components/upload
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-div'),
    html.Div(id='output-datatable'),
])


def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            parse_contents.df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')), sep = ";")
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            parse_contents.df = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),
        html.P("Inset X axis data"),
        dcc.Dropdown(id='xaxis-data',
                     options=[{'label':x, 'value':x} for x in parse_contents.df.columns]),
        html.P("Inset Y axis data"),
        dcc.Dropdown(id='yaxis-data',
                     options=[{'label':x, 'value':x} for x in parse_contents.df.columns]),
        html.Button(id="submit-button", children="Create Graph"),
        html.Hr(),

        dash_table.DataTable(
            data=parse_contents.df.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in parse_contents.df.columns],
            page_size=15
        ),
        dcc.Store(id='stored-data', data=parse_contents.df.to_dict('records')),

        html.Hr(),  # horizontal line

        # For debugging, display the raw contents provided by the web browser
        html.Div('Raw Content'),
        html.Pre(contents[0:200] + '...', style={
            'whiteSpace': 'pre-wrap',
            'wordBreak': 'break-all'
        })
    ])


@app.callback(Output('output-datatable', 'children'),
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'),
              State('upload-data', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children


@app.callback(Output('output-div', 'children'),
              Input('submit-button','n_clicks'),
              State('stored-data','data'),
              State('xaxis-data','value'),
              State('yaxis-data', 'value'))
def make_graphs(n, data, x_data, y_data):
    if n is None:
        return no_update
    else:
        bar_fig = px.bar(data, x=x_data, y=y_data)
        # print(data)
        return dcc.Graph(figure=bar_fig)


app.run_server(mode='external', port=8040)

Dash app running on http://127.0.0.1:8040/


In [10]:
parse_contents.df

Rekening Boekingsdatum  Rekeninguittrekselnummer  \
0    BE13 0635 9867 4739    14/04/2022                         4   
1    BE13 0635 9867 4739    14/04/2022                         4   
2    BE13 0635 9867 4739    14/04/2022                         4   
3    BE13 0635 9867 4739    14/04/2022                         4   
4    BE13 0635 9867 4739    13/04/2022                         4   
..                   ...           ...                       ...   
338  BE13 0635 9867 4739    27/09/2021                         9   
339  BE13 0635 9867 4739    27/09/2021                         9   
340  BE13 0635 9867 4739    27/09/2021                         9   
341  BE13 0635 9867 4739    27/09/2021                         9   
342  BE13 0635 9867 4739    27/09/2021                         9   

     Transactienummer Rekening tegenpartij      Naam tegenpartij bevat  \
0                 179  BE80 0018 2958 2977             Nicolas Mulliez   
1                 178  BE72 0635 1274 9816                 Eva Dewitte   
2                 177  BE16 7380 2553 5674              Chiro Dadizele   
3                 176  BE39 8335 1923 6519              Dewitte Wouter   
4                 175                  NaN  FRITUUR MISTRAL        LED   
..                ...                  ...                         ...   
338               228                  NaN  ABDUL WAHIDI           GEN   
339               227                  NaN  EXTREME  BVBA          ANT   
340               226  BE39 8335 1923 6519              Dewitte Wouter   
341               225                  NaN  10-000249_GENT         GEN   
342               224  BE42 0689 0209 3054          frituur t boeferke   

               Straat en nummer Postcode en plaats  \
0                           NaN                NaN   
1            Bakkerhoekstraat 7      8890 Dadizele   
2                           NaN                NaN   
3    BAKKERHOEKSTRAAT         7    8890  MOORSLEDE   
4                           NaN                NaN   
..                          ...                ...   
338                         NaN                NaN   
339                         NaN                NaN   
340  BAKKERHOEKSTRAAT         7    8890  MOORSLEDE   
341                         NaN                NaN   
342                         NaN                NaN   

                                            Transactie Valutadatum  Bedrag  \
0    OVERSCHRIJVING BELFIUS MOBILE NAAR BE80 0018 2...  14/04/2022  -14,14   
1    OVERSCHRIJVING BELFIUS MOBILE NAAR BE72 0635 1...  14/04/2022  -15,00   
2    OVERSCHRIJVING BELFIUS MOBILE NAAR BE16 7380 2...  14/04/2022  -58,02   
3    STORTING VAN BE39 8335 1923 6519 Dewitte Woute...  13/04/2022  100,00   
4    AANKOOP VIA INTERNET MET KAART NR 6703 0516 50...  13/04/2022  -19,00   
..                                                 ...         ...     ...   
338  AANKOOP BANCONTACT CONTACTLESS MET KAART NR 67...  27/09/2021  -25,00   
339  AANKOOP BANCONTACT CONTACTLESS MET KAART NR 67...  27/09/2021  -90,00   
340  STORTING VAN BE39 8335 1923 6519 Dewitte Woute...  27/09/2021  150,00   
341  GELDOPNEMING MET KAART NR 6703 0516 5093 0100 ...  27/09/2021  -40,00   
342  OVERSCHRIJVING E-COMMERCE NAAR BE42 0689 0209 ...  25/09/2021  -13,90   

    Devies       BIC Landcode  \
0      EUR  GEBABEBB      NaN   
1      EUR  GKCCBEBB       BE   
2      EUR  KREDBEBB      NaN   
3      EUR  GKCCBEBB       BE   
4      EUR       NaN      NaN   
..     ...       ...      ...   
338    EUR       NaN      NaN   
339    EUR       NaN      NaN   
340    EUR  GKCCBEBB       BE   
341    EUR       NaN      NaN   
342    EUR  GKCCBEBB      NaN   

                                          Mededelingen  
0                                             tricount  
1                                                 iris  
2                                                 poef  
3                                                  NaN  
4    AANKOOP VIA INTERNET MET KAART NR 670